# Introduction

This is a copy of the Notebook in the state it was in for our research group on Thursday, April 7th. It's definitely unfinished, but you're welcome to look through it, and the notes from the group are included below. 



## I.1. Weekly Research Group, 4/7/22

**HuggingFace `finetune_rag.py` Script**

* Script is [here](https://github.com/huggingface/transformers/blob/main/examples/research_projects/rag/finetune_rag.py).

* Command-line tool, main purpose is distributed training 
    * (distributed retrieval in particular).

* Built on PyTorch Lightning.
    * You define a "Module" class that must implement some key methods for training.

* The script mainly implements the usual training and test steps--the model implementation is really down in `RagTokenForGeneration` and `BartForConditionalGeneration`

* Fine-tuning example is under `research`.


**Process**

* Had trouble getting the tool to use our GoT dataset.
* To inspect and organize the code, I:
    * Dump the script contents into a Notebook
    * Fake the command line arguments
    * Flatten the code



**Trivia Performance**

* Doesn't appear to be tripped up by fill-in-the-blank or "implied" questions.
* Don't understand yet how the decoding style changes after fine-tuning.
    * **TODO** - Try stripping the capitalization from my training "targets"--see if the model produces lower case answers instead.
* Fine-tuning is currently making it worse--there may be bugs.
    * After fine-tuning, there are questions where it is getting the "fill-in-the-blank" version correct, but the proper question format wrong
        * So... Maybe it's unlearning the question format in favor of fill-in-the-blank
            * **TODO** - Try removing / fixing the fill-in-the-blank questions and see if it does better!

_More Observations_

* If we reformatted our answers as complete sentences, would RAG learn to do the same?
* We tried a couple easy yes or no questions that it failed on...
* We tried a couple "why" (give an explanation) type questions, and it got them wrong, but did answer in the right format (e.g., "because of a technical error").
* Transformers have some resilience to misspellings because of subwords. Mareen vs. Meereen -- Both share the "reen" subword, which can encode some information about the city.




**Trivia Question Sources**

* Most aren't easily downloadable.
* Crafting my own from articles is slower than expected! (1 minute each??)
    * Copy-pasting individually from websites is probably faster.
* Interesting that many questions would involve reasoning:
    * “Give the reason for xyz”
    * “Why was Robb motivated to revolt against xyz” 
    * “Why does Hodor say “Hodor”?”




**Summarization / Paraphrasing**

* [Chris] - I believe RAG authors report improvements on summarization benchmarks?
* [Chris] - Maybe retrieval can help in that the retriever could decide which passages from a large document are most helpful to the summarization.
    * RAG is able to incorporate multiple shorter length passages (to get around Transformer input length limitations) into its generated result.

**Next Steps**

* Use a consistent question form (remove any fill-in-the-blanks or implied questions) and see if it performs better.
    * Maybe remove periods, since those are inconsistent as well.
* Try formatting the questions as complete sentences to see how well it adapts to generating in that format.
* Work on scoring...
    * Currently does exact match.




# S1. Setup & Datasets

## 1.1. Install Packages

In [ ]:
!pip install wget

Retrieve the `requirements.txt` file which specifies the packages (and their specific version numbers) to install.

In [ ]:
import wget

wget.download("https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/rag/requirements.txt")

'requirements.txt'

Install `transformers` and other packages required for this example.

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 8.6 MB 15.1 MB/s 
     |████████████████████████████████| 325 kB 56.1 MB/s 
     |████████████████████████████████| 280 kB 55.1 MB/s 
     |████████████████████████████████| 52.7 MB 134 kB/s 
     |████████████████████████████████| 582 kB 61.3 MB/s 
     |████████████████████████████████| 4.0 MB 61.7 MB/s 
     |████████████████████████████████| 181 kB 55.1 MB/s 
     |████████████████████████████████| 212 kB 17.4 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 1.1 MB 43.7 MB/s 
     |████████████████████████████████| 136 kB 6.2 MB/s 
     |████████████████████████████████| 4.1 MB 47.3 MB/s 
     |████████████████████████████████| 226 kB 65.5 MB/s 
     |████████████████████████████████| 596 kB 58.8 MB/s 
     |████████████████████████████████| 398 kB 15.6 MB/s 
     |████████████████████████████████| 127 kB 76.1 MB/s 
     |████████████████████████████████| 6.5 MB 57.0 MB/s 
     |███████████

## 1.2. Load Reference Text Dataset

In [ ]:
import gdown

# Specify the name to give the file locally. 
output = 'got_hf_dataset.zip'
    
# Specify the Google Drive ID of the file.
file_id = '1VCCeTBgcCXaJOEupGt--1L8AAVeDeY_a' # This is currently called 'articles.zip'

# Download the file.
gdown.download('https://drive.google.com/uc?id=' + file_id, output, 
                quiet=False)

print('DONE.')

Access denied with the following error:
DONE.



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1VCCeTBgcCXaJOEupGt--1L8AAVeDeY_a 



In [ ]:
# This is the reference text dataset.
!cp './drive/Shared drives/ChrisMcCormick.AI/Content/2021-08-20 - RAG/release/got_hf_dataset.zip' .

Unzip the file.

In [ ]:
import os 

data_dir = '.'

# Check out the sizes on the saved files.
files = list(os.listdir(data_dir))

print(data_dir)

# For each file in the directory...
for f in files:
    # Get the file size, in MB
    f_size = float(os.stat(data_dir + '/' + f).st_size) / 2**20
    
    # Print the filename and its size.
    print("     {:25s}    {:>6.2f} MB".format(f, f_size))


.
     .config                        0.00 MB
     got_hf_dataset.zip           125.15 MB
     drive                          0.00 MB
     requirements.txt               0.00 MB
     sample_data                    0.00 MB


In [ ]:
#!unzip got_hf_dataset.zip -d got_hf_dataset

# -o will force it to overwrite what's there.
!unzip -o got_hf_dataset.zip

Archive:  got_hf_dataset.zip
   creating: got_dataset/
  inflating: got_dataset/state.json  
  inflating: got_dataset/dataset.arrow  
  inflating: got_dataset/dataset_info.json  


Load it in as a huggingface dataset object.

In [ ]:
from datasets import load_from_disk

got_dataset = load_from_disk("./got_dataset")


In [ ]:
got_dataset

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 43151
})

Add the required index.

In [ ]:
import faiss

# "The dimension of the embeddings to pass to the HNSW Faiss index."
dim = 768

# "The number of bi-directional links created for every new element during the 
# HNSW index construction."
m = 128

# Let's use the Faiss implementation of HNSW for fast approximate nearest neighbor search
index = faiss.IndexHNSWFlat(dim, m, faiss.METRIC_INNER_PRODUCT)

got_dataset.add_faiss_index(column="embeddings", index_name="embeddings", custom_index=index, faiss_verbose=True)

  0%|          | 0/44 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 43151
})

## 1.3. Load Trivia Questions

In [ ]:
# This is the reference text dataset.
!cp './drive/Shared drives/ChrisMcCormick.AI/Content/2021-08-20 - RAG/release/dataset_130.zip' .

In [ ]:
!unzip dataset_130.zip


Archive:  dataset_130.zip
  inflating: dataset_130/test.source  
  inflating: dataset_130/test.target  
  inflating: dataset_130/train.source  
  inflating: dataset_130/train.target  
  inflating: dataset_130/val.source  
  inflating: dataset_130/val.target  


In [ ]:
!mv ./dataset_130/ ./data/

In [ ]:
!mv ./dataset/ ./data/

mv: cannot stat './dataset/': No such file or directory


In [ ]:
!ls

data		 drive	      got_hf_dataset.zip  sample_data
dataset_130.zip  got_dataset  requirements.txt


# S2. Function & Class Definitions

## 2.1. "utils_rag.py"

In [ ]:
import itertools
import json
import linecache
import os
import pickle
import re
import socket
import string
from collections import Counter
from logging import getLogger
from pathlib import Path
from typing import Callable, Dict, Iterable, List

import git
import torch
from torch.utils.data import Dataset

from transformers import BartTokenizer, RagTokenizer, T5Tokenizer



In [ ]:

def encode_line(tokenizer, line, max_length, padding_side, pad_to_max_length=True, return_tensors="pt"):
    
    extra_kw = {"add_prefix_space": True} if isinstance(tokenizer, BartTokenizer) and not line.startswith(" ") else {}
    
    tokenizer.padding_side = padding_side
    
    return tokenizer(
        [line],
        max_length=max_length,
        padding="max_length" if pad_to_max_length else None,
        truncation=True,
        return_tensors=return_tensors,
        add_special_tokens=True,
        **extra_kw,
    )



In [ ]:

def trim_batch(
    input_ids,
    pad_token_id,
    attention_mask=None,
):
    """Remove columns that are populated exclusively by pad_token_id"""
    keep_column_mask = input_ids.ne(pad_token_id).any(dim=0)
    if attention_mask is None:
        return input_ids[:, keep_column_mask]
    else:
        return (input_ids[:, keep_column_mask], attention_mask[:, keep_column_mask])



In [ ]:

class Seq2SeqDataset(Dataset):
    def __init__(
        self,
        tokenizer,
        data_dir,
        max_source_length,
        max_target_length,
        type_path="train",
        n_obs=None,
        src_lang=None,
        tgt_lang=None,
        prefix="",
    ):
        super().__init__()
        self.src_file = Path(data_dir).joinpath(type_path + ".source")
        self.tgt_file = Path(data_dir).joinpath(type_path + ".target")
        self.src_lens = self.get_char_lens(self.src_file)
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        assert min(self.src_lens) > 0, f"found empty line in {self.src_file}"
        self.tokenizer = tokenizer
        self.prefix = prefix
        if n_obs is not None:
            self.src_lens = self.src_lens[:n_obs]
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

    def __len__(self):
        return len(self.src_lens)

    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        index = index + 1  # linecache starts at 1
        
        source_line = self.prefix + linecache.getline(str(self.src_file), index).rstrip("\n")
        
        tgt_line = linecache.getline(str(self.tgt_file), index).rstrip("\n")
        
        assert source_line, f"empty source line for index {index}"
        assert tgt_line, f"empty tgt line for index {index}"

        # Need to add eos token manually for T5
        if isinstance(self.tokenizer, T5Tokenizer):
            source_line += self.tokenizer.eos_token
            tgt_line += self.tokenizer.eos_token

        # Get the tokenizers...
        # Pad source and target to the right
        source_tokenizer = (
            self.tokenizer.question_encoder if isinstance(self.tokenizer, RagTokenizer) else self.tokenizer
        )
        target_tokenizer = self.tokenizer.generator if isinstance(self.tokenizer, RagTokenizer) else self.tokenizer

        # Encode input and desired output.
        source_inputs = encode_line(source_tokenizer, source_line, self.max_source_length, "right")
        
        target_inputs = encode_line(target_tokenizer, tgt_line, self.max_target_length, "right")

        source_ids = source_inputs["input_ids"].squeeze()

        target_ids = target_inputs["input_ids"].squeeze()

        src_mask = source_inputs["attention_mask"].squeeze()

        return {
            "input_ids": source_ids,
            "attention_mask": src_mask,
            "decoder_input_ids": target_ids,
        }

    @staticmethod
    def get_char_lens(data_file):
        return [len(x) for x in Path(data_file).open().readlines()]

    def collate_fn(self, batch) -> Dict[str, torch.Tensor]:
        input_ids = torch.stack([x["input_ids"] for x in batch])
        masks = torch.stack([x["attention_mask"] for x in batch])
        target_ids = torch.stack([x["decoder_input_ids"] for x in batch])
        tgt_pad_token_id = (
            self.tokenizer.generator.pad_token_id
            if isinstance(self.tokenizer, RagTokenizer)
            else self.tokenizer.pad_token_id
        )
        src_pad_token_id = (
            self.tokenizer.question_encoder.pad_token_id
            if isinstance(self.tokenizer, RagTokenizer)
            else self.tokenizer.pad_token_id
        )
        y = trim_batch(target_ids, tgt_pad_token_id)
        source_ids, source_mask = trim_batch(input_ids, src_pad_token_id, attention_mask=masks)
        batch = {
            "input_ids": source_ids,
            "attention_mask": source_mask,
            "decoder_input_ids": y,
        }
        return batch



In [ ]:

logger = getLogger(__name__)


def flatten_list(summary_ids: List[List]):
    return [x for x in itertools.chain.from_iterable(summary_ids)]


def save_git_info(folder_path: str) -> None:
    """Save git information to output_dir/git_log.json"""
    repo_infos = get_git_info()
    save_json(repo_infos, os.path.join(folder_path, "git_log.json"))


def save_json(content, path, indent=4, **json_dump_kwargs):
    with open(path, "w") as f:
        json.dump(content, f, indent=indent, **json_dump_kwargs)


def load_json(path):
    with open(path) as f:
        return json.load(f)


def get_git_info():
    repo = git.Repo(search_parent_directories=True)
    repo_infos = {
        "repo_id": str(repo),
        "repo_sha": str(repo.head.object.hexsha),
        "repo_branch": str(repo.active_branch),
        "hostname": str(socket.gethostname()),
    }
    return repo_infos


def lmap(f: Callable, x: Iterable) -> List:
    """list(map(f, x))"""
    return list(map(f, x))


def pickle_save(obj, path):
    """pickle.dump(obj, path)"""
    with open(path, "wb") as f:
        return pickle.dump(obj, f)


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def calculate_exact_match(output_lns: List[str], reference_lns: List[str]) -> Dict:
    
    assert len(output_lns) == len(reference_lns)
    
    em = 0
    
    # For each token, count the number which match exactly.
    for hypo, pred in zip(output_lns, reference_lns):
        # Do a little cleaning of each string, but then check for exact 
        # string match.
        em += (normalize_answer(hypo) == normalize_answer(pred))
    
    # Average the total exact matches over the sequence length.
    if len(output_lns) > 0:
        em /= len(output_lns)

    return {"em": em}

def set_extra_model_params(extra_params, hparams, config):
    equivalent_param = {p: p for p in extra_params}
    # T5 models don't have `dropout` param, they have `dropout_rate` instead
    equivalent_param["dropout"] = "dropout_rate"
    for p in extra_params:
        if getattr(hparams, p, None):
            if not hasattr(config, p) and not hasattr(config, equivalent_param[p]):
                logger.info("config doesn't have a `{}` attribute".format(p))
                delattr(hparams, p)
                continue
            set_p = p if hasattr(config, p) else equivalent_param[p]
            setattr(config, set_p, getattr(hparams, p))
            delattr(hparams, p)
    return hparams, config

## 2.2. "callbacks_rag.py"

In [ ]:
import logging
from pathlib import Path

import numpy as np
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.utilities import rank_zero_only

#from utils_rag import save_json


def count_trainable_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params


logger = logging.getLogger(__name__)


def get_checkpoint_callback(output_dir, metric):
    """Saves the best model by validation EM score."""
    if metric == "rouge2":
        exp = "{val_avg_rouge2:.4f}-{step_count}"
    elif metric == "bleu":
        exp = "{val_avg_bleu:.4f}-{step_count}"
    elif metric == "em":
        exp = "{val_avg_em:.4f}-{step_count}"
    else:
        raise NotImplementedError(
            f"seq2seq callbacks only support rouge2 and bleu, got {metric}, You can make your own by adding to this function."
        )

    checkpoint_callback = ModelCheckpoint(
        dirpath=output_dir,
        filename=exp,
        monitor=f"val_{metric}",
        mode="max",
        save_top_k=3,
        every_n_epochs=1,  # maybe save a checkpoint every time val is run, not just end of epoch.
    )
    return checkpoint_callback


def get_early_stopping_callback(metric, patience):
    return EarlyStopping(
        monitor=f"val_{metric}",  # does this need avg?
        mode="min" if "loss" in metric else "max",
        patience=patience,
        verbose=True,
    )


class Seq2SeqLoggingCallback(pl.Callback):
    def on_batch_end(self, trainer, pl_module):
        lrs = {f"lr_group_{i}": param["lr"] for i, param in enumerate(pl_module.trainer.optimizers[0].param_groups)}
        pl_module.logger.log_metrics(lrs)

    @rank_zero_only
    def _write_logs(
        self, trainer: pl.Trainer, pl_module: pl.LightningModule, type_path: str, save_generations=True
    ) -> None:
        logger.info(f"***** {type_path} results at step {trainer.global_step:05d} *****")
        metrics = trainer.callback_metrics
        trainer.logger.log_metrics({k: v for k, v in metrics.items() if k not in ["log", "progress_bar", "preds"]})
        # Log results
        od = Path(pl_module.hparams.output_dir)
        if type_path == "test":
            results_file = od / "test_results.txt"
            generations_file = od / "test_generations.txt"
        else:
            # this never gets hit. I prefer not to save intermediate generations, and results are in metrics.json
            # If people want this it will be easy enough to add back.
            results_file = od / f"{type_path}_results/{trainer.global_step:05d}.txt"
            generations_file = od / f"{type_path}_generations/{trainer.global_step:05d}.txt"
            results_file.parent.mkdir(exist_ok=True)
            generations_file.parent.mkdir(exist_ok=True)
        with open(results_file, "a+") as writer:
            for key in sorted(metrics):
                if key in ["log", "progress_bar", "preds"]:
                    continue
                val = metrics[key]
                if isinstance(val, torch.Tensor):
                    val = val.item()
                msg = f"{key}: {val:.6f}\n"
                writer.write(msg)

        if not save_generations:
            return

        if "preds" in metrics:
            content = "\n".join(metrics["preds"])
            generations_file.open("w+").write(content)

    @rank_zero_only
    def on_train_start(self, trainer, pl_module):
        try:
            npars = pl_module.model.model.num_parameters()
        except AttributeError:
            npars = pl_module.model.num_parameters()

        n_trainable_pars = count_trainable_parameters(pl_module)
        # mp stands for million parameters
        trainer.logger.log_metrics({"n_params": npars, "mp": npars / 1e6, "grad_mp": n_trainable_pars / 1e6})

    @rank_zero_only
    def on_test_end(self, trainer: pl.Trainer, pl_module: pl.LightningModule):
        save_json(pl_module.metrics, pl_module.metrics_save_path)
        return self._write_logs(trainer, pl_module, "test")

    @rank_zero_only
    def on_validation_end(self, trainer: pl.Trainer, pl_module):
        save_json(pl_module.metrics, pl_module.metrics_save_path)
        # Uncommenting this will save val generations
        # return self._write_logs(trainer, pl_module, "valid")

## 2.3. "lightning_base.py"

In [ ]:
import argparse
import logging
import os
from pathlib import Path
from typing import Any, Dict

import pytorch_lightning as pl
from pytorch_lightning.utilities import rank_zero_info

from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoModelForPreTraining,
    AutoModelForQuestionAnswering,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    AutoModelWithLMHead,
    AutoTokenizer,
    PretrainedConfig,
    PreTrainedTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_cosine_schedule_with_warmup,
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_polynomial_decay_schedule_with_warmup,
)
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

require_version("pytorch_lightning>=1.0.4")

MODEL_MODES = {
    "base": AutoModel,
    "sequence-classification": AutoModelForSequenceClassification,
    "question-answering": AutoModelForQuestionAnswering,
    "pretraining": AutoModelForPreTraining,
    "token-classification": AutoModelForTokenClassification,
    "language-modeling": AutoModelWithLMHead,
    "summarization": AutoModelForSeq2SeqLM,
    "translation": AutoModelForSeq2SeqLM,
}


# update this and the import above to support new schedulers from transformers.optimization
arg_to_scheduler = {
    "linear": get_linear_schedule_with_warmup,
    "cosine": get_cosine_schedule_with_warmup,
    "cosine_w_restarts": get_cosine_with_hard_restarts_schedule_with_warmup,
    "polynomial": get_polynomial_decay_schedule_with_warmup,
    # '': get_constant_schedule,             # not supported for now
    # '': get_constant_schedule_with_warmup, # not supported for now
}
arg_to_scheduler_choices = sorted(arg_to_scheduler.keys())
arg_to_scheduler_metavar = "{" + ", ".join(arg_to_scheduler_choices) + "}"


### 2.3.1. Class: BaseTransformer

In [ ]:
class BaseTransformer(pl.LightningModule):
    def __init__(
        self,
        hparams: argparse.Namespace,
        num_labels=None,
        mode="base",
        config=None,
        tokenizer=None,
        model=None,
        **config_kwargs
    ):
        """Initialize a model, tokenizer and config."""
        super().__init__()
        # TODO: move to self.save_hyperparameters()
        # self.save_hyperparameters()
        # can also expand arguments into trainer signature for easier reading

        self.save_hyperparameters(hparams)
        self.step_count = 0
        self.output_dir = Path(self.hparams.output_dir)
        cache_dir = self.hparams.cache_dir if self.hparams.cache_dir else None
        if config is None:
            self.config = AutoConfig.from_pretrained(
                self.hparams.config_name if self.hparams.config_name else self.hparams.model_name_or_path,
                **({"num_labels": num_labels} if num_labels is not None else {}),
                cache_dir=cache_dir,
                **config_kwargs,
            )
        else:
            self.config: PretrainedConfig = config

        extra_model_params = ("encoder_layerdrop", "decoder_layerdrop", "dropout", "attention_dropout")
        for p in extra_model_params:
            if getattr(self.hparams, p, None):
                assert hasattr(self.config, p), f"model config doesn't have a `{p}` attribute"
                setattr(self.config, p, getattr(self.hparams, p))

        if tokenizer is None:
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.hparams.tokenizer_name if self.hparams.tokenizer_name else self.hparams.model_name_or_path,
                cache_dir=cache_dir,
            )
        else:
            self.tokenizer: PreTrainedTokenizer = tokenizer
        self.model_type = MODEL_MODES[mode]
        if model is None:
            self.model = self.model_type.from_pretrained(
                self.hparams.model_name_or_path,
                from_tf=bool(".ckpt" in self.hparams.model_name_or_path),
                config=self.config,
                cache_dir=cache_dir,
            )
        else:
            self.model = model

    def load_hf_checkpoint(self, *args, **kwargs):
        self.model = self.model_type.from_pretrained(*args, **kwargs)

    def get_lr_scheduler(self):
        get_schedule_func = arg_to_scheduler[self.hparams.lr_scheduler]
        scheduler = get_schedule_func(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=self.total_steps()
        )
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return scheduler

    def configure_optimizers(self):
        """Prepare optimizer and schedule (linear warmup and decay)"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        if self.hparams.adafactor:
            optimizer = Adafactor(
                optimizer_grouped_parameters, lr=self.hparams.learning_rate, scale_parameter=False, relative_step=False
            )

        else:
            optimizer = AdamW(
                optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon
            )
        self.opt = optimizer

        scheduler = self.get_lr_scheduler()

        return [optimizer], [scheduler]

    def test_step(self, batch, batch_nb):
        return self.validation_step(batch, batch_nb)

    def test_epoch_end(self, outputs):
        return self.validation_end(outputs)

    def total_steps(self) -> int:
        """The number of total training steps that will be run. Used for lr scheduler purposes."""
        num_devices = max(1, self.hparams.gpus)  # TODO: consider num_tpu_cores
        effective_batch_size = self.hparams.train_batch_size * self.hparams.accumulate_grad_batches * num_devices
        return (self.dataset_size / effective_batch_size) * self.hparams.max_epochs

    def setup(self, stage):
        if stage == "test":
            self.dataset_size = len(self.test_dataloader().dataset)
        else:
            self.train_loader = self.get_dataloader("train", self.hparams.train_batch_size, shuffle=True)
            self.dataset_size = len(self.train_dataloader().dataset)

    def get_dataloader(self, type_path: str, batch_size: int, shuffle: bool = False):
        raise NotImplementedError("You must implement this for your task")

    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.get_dataloader("dev", self.hparams.eval_batch_size, shuffle=False)

    def test_dataloader(self):
        return self.get_dataloader("test", self.hparams.eval_batch_size, shuffle=False)

    def _feature_file(self, mode):
        return os.path.join(
            self.hparams.data_dir,
            "cached_{}_{}_{}".format(
                mode,
                list(filter(None, self.hparams.model_name_or_path.split("/"))).pop(),
                str(self.hparams.max_seq_length),
            ),
        )

    @pl.utilities.rank_zero_only
    def on_save_checkpoint(self, checkpoint: Dict[str, Any]) -> None:
        save_path = self.output_dir.joinpath("best_tfmr")
        self.model.config.save_step = self.step_count
        self.model.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)


In [ ]:


class InitCallback(pl.Callback):
    # This method is better that using a custom DDP plugging with the latest pytorch-lightning (@shamanez)
    def on_sanity_check_start(self, trainer, pl_module):
        
        # https://github.com/huggingface/transformers/master/src/transformers/models/rag/retrieval_rag.py#L458
        pl_module.model.rag.retriever.init_retrieval()  # better to use hook functions.



In [ ]:

class LoggingCallback(pl.Callback):
    def on_batch_end(self, trainer, pl_module):
        lr_scheduler = trainer.lr_schedulers[0]["scheduler"]
        lrs = {f"lr_group_{i}": lr for i, lr in enumerate(lr_scheduler.get_lr())}
        pl_module.logger.log_metrics(lrs)

    def on_validation_end(self, trainer: pl.Trainer, pl_module: pl.LightningModule):
        rank_zero_info("***** Validation results *****")
        metrics = trainer.callback_metrics
        # Log results
        for key in sorted(metrics):
            if key not in ["log", "progress_bar"]:
                rank_zero_info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer: pl.Trainer, pl_module: pl.LightningModule):
        rank_zero_info("***** Test results *****")
        metrics = trainer.callback_metrics
        # Log and save results to file
        output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
        with open(output_test_results_file, "w") as writer:
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    rank_zero_info("{} = {}\n".format(key, str(metrics[key])))
                    writer.write("{} = {}\n".format(key, str(metrics[key])))



### 2.3.2. "generic_train"

In [ ]:

def generic_train(
    model: BaseTransformer,
    args: argparse.Namespace,
    early_stopping_callback=None,
    logger=True,  # can pass WandbLogger() here
    custom_ddp_plugin=None,
    extra_callbacks=[],
    checkpoint_callback=None,
    logging_callback=None,
    **extra_train_kwargs
):
    pl.seed_everything(args.seed)

    # init model
    odir = Path(model.hparams.output_dir)
    odir.mkdir(exist_ok=True)

    # add custom checkpoints
    if checkpoint_callback is None:
        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=1
        )
    if early_stopping_callback:
        extra_callbacks.append(early_stopping_callback)
    if logging_callback is None:
        logging_callback = LoggingCallback()

    train_params = {}

    # TODO: remove with PyTorch 1.6 since pl uses native amp
    if args.fp16:
        train_params["precision"] = 16
        # train_params["amp_level"] = args.fp16_opt_level

    if args.gpus > 1:
        train_params["accelerator"] = "auto"  # "ddp"
        train_params["strategy"] = "ddp"

    train_params["accumulate_grad_batches"] = args.accumulate_grad_batches
    train_params["profiler"] = None  # extra_train_kwargs.get("profiler", None) #get unwanted logs
    train_params["devices"] = "auto"

    trainer = pl.Trainer.from_argparse_args(
        args,
        weights_summary=None,
        # [Chris] - Trying to disable checkpointing to work around an issue with the validation metrics...
        #callbacks=[logging_callback] + extra_callbacks + [checkpoint_callback] + [InitCallback()],
        callbacks=[logging_callback] + extra_callbacks + [InitCallback()],
        # plugins=[custom_ddp_plugin],
        logger=logger,
        **train_params,
    )

    if args.do_train:
        trainer.fit(model)

    return trainer


## 2.4. "finetune_rag.py"

These are all standard libraries, plus `transformers`.

In [ ]:
"""Finetuning script for RAG models. Adapted from examples.seq2seq.finetune.py"""

import argparse
import logging
import os
import sys
import time
from collections import defaultdict
from pathlib import Path
from typing import Any, Dict, List, Tuple

import numpy as np
import pytorch_lightning as pl
import torch
import torch.distributed as dist
import torch.distributed as torch_distrib
from pytorch_lightning.plugins.training_type import DDPPlugin
from torch.utils.data import DataLoader

from transformers import (
    AutoConfig,
    AutoTokenizer,
    BartForConditionalGeneration,
    BatchEncoding,
    RagConfig,
    RagSequenceForGeneration,
    RagTokenForGeneration,
    RagTokenizer,
    T5ForConditionalGeneration,
)

from transformers import logging as transformers_logging



In [ ]:
# [Chris] - Need to fake this variable. The expression below just gives '/'
#__file__ = "/content/finetune_rag.py"

# need the parent dir module
#sys.path.insert(2, str(Path(__file__).resolve().parents[1]))

sys.path.insert(2, '/')

Set up logging.

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

transformers_logging.set_verbosity_info()



In [ ]:

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self



### Class: GenerativeQAModule

This class is an example of a LightningModule, which is what you define in order to use the PyTorch Lightning Trainer.

The documentation is here: https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html

There are really two key functions here:

**_step**
* This performs a forward pass of the whole architecture and calculates the loss for the purpose of training.
   * However, the real forward pass is implemented in `RagTokenForGeneration`-- there's very little actually here!

**_generative_step**
* This also runs through the whole architecture, but generates the final text output.

These two operations are not the same--RAG does something different on the output when we're training it.


In [ ]:
from transformers import RagRetriever

class GenerativeQAModule(BaseTransformer):
    mode = "generative_qa"
    loss_names = ["loss"]
    metric_names = ["em"]
    val_metric = "em"


    def __init__(self, hparams, config, tokenizer, model, **kwargs):
        
        # when loading from a pytorch lightning checkpoint, hparams are passed as dict
        if isinstance(hparams, dict):
            hparams = AttrDict(hparams)

        # Presumably this function handles the self.model = model step.
        super().__init__(hparams, config=config, tokenizer=tokenizer, model=model)
        
        
        self.output_dir = Path(self.hparams.output_dir)
        self.metrics_save_path = Path(self.output_dir) / "metrics.json"
        self.hparams_save_path = Path(self.output_dir) / "hparams.pkl"
        pickle_save(self.hparams, self.hparams_save_path)
        self.step_count = 0
        self.metrics = defaultdict(list)

        self.dataset_kwargs: dict = dict(
            data_dir=self.hparams.data_dir,
            max_source_length=self.hparams.max_source_length,
            #prefix = prefix or "",
            prefix = ""
        )

        n_observations_per_split = {
            "train": self.hparams.n_train,
            "val": self.hparams.n_val,
            "test": self.hparams.n_test,
        }

        self.n_obs = {k: v if v >= 0 else None for k, v in n_observations_per_split.items()}

        self.target_lens = {
            "train": self.hparams.max_target_length,
            "val": self.hparams.val_max_target_length,
            "test": self.hparams.test_max_target_length,
        }
        assert self.target_lens["train"] <= self.target_lens["val"], f"target_lens: {self.target_lens}"
        assert self.target_lens["train"] <= self.target_lens["test"], f"target_lens: {self.target_lens}"

        # [Chris] - Removing dependencies to git.
        #self.hparams.git_sha = get_git_info()["repo_sha"]
        
        self.num_workers = hparams.num_workers
        self.distributed_port = self.hparams.distributed_port

        # For single GPU training, init_ddp_connection is not called.
        # So we need to initialize the retrievers here.
        if False:
            if hparams.gpus <= 1:
                if hparams.distributed_retriever == "ray":
                    self.model.retriever.init_retrieval()
                elif hparams.distributed_retriever == "pytorch":
                    self.model.retriever.init_retrieval(self.distributed_port)

            self.distributed_retriever = hparams.distributed_retriever


    # ======== forward ========
    # Docs: "Use for inference only (separate from training_step)"
    #
    def forward(self, input_ids, **kwargs):
        return self.model(input_ids, **kwargs)


    # ======== ids_to_clean_text ========
    # TODO - I'm unclear on whether this is intended to be called on a single
    #        output sequence or a batch...
    # 
    def ids_to_clean_text(self, generated_ids: List[int]):
        
        # self.tokenizer is of type RagTokenizer.
        # RagTokenizer.batch_decode just calls down to `BartTokenizerFast.batch_decode`.
        gen_text = self.tokenizer.batch_decode(
            generated_ids, 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=True
        )
        
        # lmap will call 'strip' on each element of 'gen_text', and returns
        # the results as a list.
        return lmap(str.strip, gen_text)


    # ======== _step ========
    # [Chris] - This doesn't appear to be lightning...
    # Both '_generative_step' and 'training_step' call this.
    # 
    # 1. 'training_step' just calls this and adds some logging afterwards.
    # 
    # 2. '_generative_step' (which is the implementation of both validation and 
    #    test), 
    #  
    def _step(self, batch: dict) -> Tuple:
        source_ids, source_mask, target_ids = batch["input_ids"], batch["attention_mask"], batch["decoder_input_ids"]

        rag_kwargs = {}
        
        generator = self.model.rag.generator
        
        decoder_input_ids = target_ids
        
        lm_labels = decoder_input_ids
        rag_kwargs["reduce_loss"] = True

        assert decoder_input_ids is not None

        outputs = self(
            source_ids,
            attention_mask=source_mask,
            decoder_input_ids=decoder_input_ids,
            use_cache=False,
            labels=lm_labels,
            **rag_kwargs,
        )

        loss = outputs["loss"]
        return (loss,)

    @property
    def pad(self) -> int:
        raise NotImplementedError("pad not implemented")

    # ======== training_step ========
    # Lightning function. Doc: "the complete training loop"
    def training_step(self, batch, batch_idx) -> Dict:
        loss_tensors = self._step(batch)

        logs = {name: loss.detach() for name, loss in zip(self.loss_names, loss_tensors)}
        # tokens per batch
        tgt_pad_token_id = (
            self.tokenizer.generator.pad_token_id
            if isinstance(self.tokenizer, RagTokenizer)
            else self.tokenizer.pad_token_id
        )
        src_pad_token_id = (
            self.tokenizer.question_encoder.pad_token_id
            if isinstance(self.tokenizer, RagTokenizer)
            else self.tokenizer.pad_token_id
        )
        logs["tpb"] = (
            batch["input_ids"].ne(src_pad_token_id).sum() + batch["decoder_input_ids"].ne(tgt_pad_token_id).sum()
        )

        return {"loss": loss_tensors[0], "log": logs}

    # ======== validation_step ========
    # Ligntning function. Doc: "the complete validation loop"
    def validation_step(self, batch, batch_idx) -> Dict:
        return self._generative_step(batch)

    # ======== validation_epoch_end ========
    #
    def validation_epoch_end(self, outputs, prefix="val") -> Dict:
        self.step_count += 1
        losses = {k: torch.stack([x[k] for x in outputs]).mean() for k in self.loss_names}
        loss = losses["loss"]
        gen_metrics = {
            k: np.array([x[k] for x in outputs]).mean() for k in self.metric_names + ["gen_time", "gen_len"]
        }
        metrics_tensor: torch.FloatTensor = torch.tensor(gen_metrics[self.val_metric]).type_as(loss)
        gen_metrics.update({k: v.item() for k, v in losses.items()})

        # fix for https://github.com/PyTorchLightning/pytorch-lightning/issues/2424
        if dist.is_initialized():
            dist.all_reduce(metrics_tensor, op=dist.ReduceOp.SUM)
            metrics_tensor = metrics_tensor / dist.get_world_size()
            gen_metrics.update({self.val_metric: metrics_tensor.item()})

        losses.update(gen_metrics)
        metrics = {f"{prefix}_avg_{k}": x for k, x in losses.items()}
        metrics["step_count"] = self.step_count
        self.save_metrics(metrics, prefix)  # writes to self.metrics_save_path
        preds = flatten_list([x["preds"] for x in outputs])
        return {"log": metrics, "preds": preds, f"{prefix}_loss": loss, f"{prefix}_{self.val_metric}": metrics_tensor}

    def save_metrics(self, latest_metrics, type_path) -> None:
        self.metrics[type_path].append(latest_metrics)
        save_json(self.metrics, self.metrics_save_path)

    # ======== _generative_step ========
    # Non-lightning.
    # 'validation_step' and 'test_step' simply pass down to this funciton. 
    #
    def _generative_step(self, batch: dict) -> dict:
        
        start_time = time.time()
        
        batch = BatchEncoding(batch).to(device=self.model.device)
        
        # Run the inputs through the entire framework! 
        generated_ids = self.model.generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
            do_deduplication=False,  # rag specific parameter
            use_cache=True,
            min_length=1,
            max_length=self.target_lens["val"],
        )

        gen_time = (time.time() - start_time) / batch["input_ids"].shape[0]
        
        # Turn the generated token IDs into a list of token strings.
        preds: List[str] = self.ids_to_clean_text(generated_ids)
        
        # Also get the list of correct IDs (again, as a list of token strings).
        target: List[str] = self.ids_to_clean_text(batch["decoder_input_ids"])

        # Call an iteration of the training loop just to calculate the loss.
        loss_tensors = self._step(batch)
        base_metrics = {name: loss for name, loss in zip(self.loss_names, loss_tensors)}
        
        # Score the output! Exact match is simply the number of tokens which
        # match (and it's position dependent!)
        gen_metrics: Dict = calculate_exact_match(preds, target)

        summ_len = np.mean(lmap(len, generated_ids))
        
        base_metrics.update(gen_time=gen_time, gen_len=summ_len, preds=preds, target=target, **gen_metrics)
        
        return base_metrics

    # ======== test_step ========
    # Lightning function. Doc: "the complete test loop"
    def test_step(self, batch, batch_idx):
        return self._generative_step(batch)

    def test_epoch_end(self, outputs):
        return self.validation_epoch_end(outputs, prefix="test")

    def get_dataset(self, type_path) -> Seq2SeqDataset:
        n_obs = self.n_obs[type_path]
        max_target_length = self.target_lens[type_path]
        dataset = Seq2SeqDataset(
            self.tokenizer,
            type_path=type_path,
            n_obs=n_obs,
            max_target_length=max_target_length,
            **self.dataset_kwargs,
        )
        return dataset

    def get_dataloader(self, type_path: str, batch_size: int, shuffle: bool = False) -> DataLoader:
        dataset = self.get_dataset(type_path)

        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            collate_fn=dataset.collate_fn,
            shuffle=shuffle,
            num_workers=self.num_workers,
        )
        return dataloader

    def train_dataloader(self) -> DataLoader:
        dataloader = self.get_dataloader("train", batch_size=self.hparams.train_batch_size, shuffle=True)
        return dataloader

    def val_dataloader(self) -> DataLoader:
        return self.get_dataloader("val", batch_size=self.hparams.eval_batch_size)

    def test_dataloader(self) -> DataLoader:
        return self.get_dataloader("test", batch_size=self.hparams.eval_batch_size)

    @pl.utilities.rank_zero_only
    def on_save_checkpoint(self, checkpoint: Dict[str, Any]) -> None:
        save_path = self.output_dir.joinpath("checkpoint{}".format(self.step_count))
        self.model.config.save_step = self.step_count
        self.model.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)





## 2.5. Argument Definitions

In [ ]:
# Create an ArgumentParser object.
parser = argparse.ArgumentParser()

In [ ]:
# Some standard arguments for PyTorch Lightning?
parser = pl.Trainer.add_argparse_args(parser)

# Pytorch Lightning Profiler
parser.add_argument(
    "--profile",
    action="store_true",
    help="If True, use pytorch_lightning.profiler.AdvancedProfiler to profile the Trainer.",
)

_StoreTrueAction(option_strings=['--profile'], dest='profile', nargs=0, const=True, default=False, type=None, choices=None, help='If True, use pytorch_lightning.profiler.AdvancedProfiler to profile the Trainer.', metavar=None)

In [ ]:
# Some of the arguments require this "root_dir" parameter.
root_dir = os.getcwd()

**BaseTransformer --> add_model_specific_args**

In [ ]:
# TODO - Needs arguments: parser, root_dir

parser.add_argument(
    "--model_name_or_path",
    default=None,
    type=str,
    required=True,
    help="Path to pretrained model or model identifier from huggingface.co/models",
)
parser.add_argument(
    "--config_name", default="", type=str, help="Pretrained config name or path if not the same as model_name"
)
parser.add_argument(
    "--tokenizer_name",
    default=None,
    type=str,
    help="Pretrained tokenizer name or path if not the same as model_name",
)
parser.add_argument(
    "--cache_dir",
    default="",
    type=str,
    help="Where do you want to store the pre-trained models downloaded from huggingface.co",
)
parser.add_argument(
    "--encoder_layerdrop",
    type=float,
    help="Encoder layer dropout probability (Optional). Goes into model.config",
)
parser.add_argument(
    "--decoder_layerdrop",
    type=float,
    help="Decoder layer dropout probability (Optional). Goes into model.config",
)
parser.add_argument(
    "--dropout",
    type=float,
    help="Dropout probability (Optional). Goes into model.config",
)
parser.add_argument(
    "--attention_dropout",
    type=float,
    help="Attention dropout probability (Optional). Goes into model.config",
)
parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
parser.add_argument(
    "--lr_scheduler",
    default="linear",
    choices=arg_to_scheduler_choices,
    metavar=arg_to_scheduler_metavar,
    type=str,
    help="Learning rate scheduler",
)
parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")
parser.add_argument("--num_workers", default=4, type=int, help="kwarg passed to DataLoader")
parser.add_argument("--num_train_epochs", dest="max_epochs", default=3, type=int)
parser.add_argument("--train_batch_size", default=32, type=int)
parser.add_argument("--eval_batch_size", default=32, type=int)
parser.add_argument("--adafactor", action="store_true")

_StoreTrueAction(option_strings=['--adafactor'], dest='adafactor', nargs=0, const=True, default=False, type=None, choices=None, help=None, metavar=None)

**add_generic_args**

In [ ]:
#  To allow all pl args uncomment the following line
#  parser = pl.Trainer.add_argparse_args(parser)
parser.add_argument(
    "--output_dir",
    default=None,
    type=str,
    required=True,
    help="The output directory where the model predictions and checkpoints will be written.",
)
parser.add_argument(
    "--fp16",
    action="store_true",
    help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
)

parser.add_argument(
    "--fp16_opt_level",
    type=str,
    default="O2",
    help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
    "See details at https://nvidia.github.io/apex/amp.html",
)
parser.add_argument("--n_tpu_cores", dest="tpu_cores", type=int)
parser.add_argument("--max_grad_norm", dest="gradient_clip_val", default=1.0, type=float, help="Max gradient norm")
parser.add_argument("--do_train", action="store_true", help="Whether to run training.")
parser.add_argument("--do_predict", action="store_true", help="Whether to run predictions on the test set.")
parser.add_argument(
    "--gradient_accumulation_steps",
    dest="accumulate_grad_batches",
    type=int,
    default=1,
    help="Number of updates steps to accumulate before performing a backward/update pass.",
)
parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")
parser.add_argument(
    "--data_dir",
    default=None,
    type=str,
    required=True,
    help="The input data dir. Should contain the training files for the CoNLL-2003 NER task.",
    )

_StoreAction(option_strings=['--data_dir'], dest='data_dir', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='The input data dir. Should contain the training files for the CoNLL-2003 NER task.', metavar=None)

**GenerativeQAModel --> add_model_specific_args**

In [ ]:
parser.add_argument(
    "--max_source_length",
    default=128,
    type=int,
    help="The maximum total input sequence length after tokenization. Sequences longer "
    "than this will be truncated, sequences shorter will be padded.",
)
parser.add_argument(
    "--max_target_length",
    default=25,
    type=int,
    help="The maximum total input sequence length after tokenization. Sequences longer "
    "than this will be truncated, sequences shorter will be padded.",
)
parser.add_argument(
    "--val_max_target_length",
    default=25,
    type=int,
    help="The maximum total input sequence length after tokenization. Sequences longer "
    "than this will be truncated, sequences shorter will be padded.",
)
parser.add_argument(
    "--test_max_target_length",
    default=25,
    type=int,
    help="The maximum total input sequence length after tokenization. Sequences longer "
    "than this will be truncated, sequences shorter will be padded.",
)
parser.add_argument("--logger_name", type=str, choices=["default", "wandb", "wandb_shared"], default="default")
parser.add_argument("--n_train", type=int, default=-1, required=False, help="# examples. -1 means use all.")
parser.add_argument("--n_val", type=int, default=-1, required=False, help="# examples. -1 means use all.")
parser.add_argument("--n_test", type=int, default=-1, required=False, help="# examples. -1 means use all.")
parser.add_argument("--label_smoothing", type=float, default=0.0, required=False)
parser.add_argument(
    "--prefix",
    type=str,
    default=None,
    help="Prefix added at the beginning of each text, typically used with T5-based models.",
)
parser.add_argument(
    "--early_stopping_patience",
    type=int,
    default=-1,
    required=False,
    help="-1 means never early stop. early_stopping_patience is measured in validation checks, not epochs. So val_check_interval will effect it.",
)
parser.add_argument(
    "--distributed-port", type=int, default=-1, required=False, help="Port number for distributed training."
)
parser.add_argument(
    "--model_type",
    choices=["rag_sequence", "rag_token", "bart", "t5"],
    type=str,
    help="RAG model type: sequence or token, if none specified, the type is inferred from the model_name_or_path",
)
        


_StoreAction(option_strings=['--model_type'], dest='model_type', nargs=None, const=None, default=None, type=<class 'str'>, choices=['rag_sequence', 'rag_token', 'bart', 't5'], help='RAG model type: sequence or token, if none specified, the type is inferred from the model_name_or_path', metavar=None)

**GenerativeQAModule --> add_retriever_specific_args**

In [ ]:
parser.add_argument(
    "--index_name",
    type=str,
    default=None,
    help="Name of the index to use: 'hf' for a canonical dataset from the datasets library (default), 'custom' for a local index, or 'legacy' for the orignal one)",
)
parser.add_argument(
    "--passages_path",
    type=str,
    default=None,
    help="Path to the dataset of passages for custom index. More info about custom indexes in the RagRetriever documentation as well as in `examples/rag/use_own_knowledge_dataset.py`",
)
parser.add_argument(
    "--index_path",
    type=str,
    default=None,
    help="Path to the faiss index for custom index. More info about custom indexes in the RagRetriever documentation as well as in `examples/rag/use_own_knowledge_dataset.py`",
)
parser.add_argument(
    "--distributed_retriever",
    choices=["ray", "pytorch"],
    type=str,
    default="pytorch",
    help="What implementation to use for distributed retriever? If "
    "pytorch is selected, the index is loaded on training "
    "worker 0, and torch.distributed is used to handle "
    "communication between training worker 0, and the other "
    "training workers. If ray is selected, the Ray library is "
    "used to create load the index on separate processes, "
    "and Ray handles the communication between the training "
    "workers and the retrieval actors.",
)
parser.add_argument(
    "--use_dummy_dataset",
    type=bool,
    default=False,
    help="Whether to use the dummy version of the dataset index. More info about custom indexes in the RagRetriever documentation as well as in `examples/rag/use_own_knowledge_dataset.py`",
)


_StoreAction(option_strings=['--use_dummy_dataset'], dest='use_dummy_dataset', nargs=None, const=None, default=False, type=<class 'bool'>, choices=None, help='Whether to use the dummy version of the dataset index. More info about custom indexes in the RagRetriever documentation as well as in `examples/rag/use_own_knowledge_dataset.py`', metavar=None)

Ray-specific arguments (unused)

In [ ]:

if False:
    def add_ray_specific_args(parser):
        # Ray cluster address.
        parser.add_argument(
            "--ray-address",
            default="auto",
            type=str,
            help="The address of the Ray cluster to connect to. If not "
            "specified, Ray will attempt to automatically detect the "
            "cluster. Has no effect if pytorch is used as the distributed "
            "retriever.",
        )
        parser.add_argument(
            "--num_retrieval_workers",
            type=int,
            default=1,
            help="The number of retrieval actors to use when Ray is selected"
            "for the distributed retriever. Has no effect when "
            "distributed_retriever is set to pytorch.",
        )
        return parser

# S3. Setup Model


## 3.1. Set Script Arguments

Set the arguments in the code rather than with a command line parser.

In [ ]:
sys.argv = ['./test_argv.py', 
            '--data_dir', './data', 
            '--output_dir', './output', 
            '--model_name_or_path', 'facebook/rag-token-base', 
            '--model_type', 'rag_token', 
            '--use_dummy_dataset', 'True', 
            '--fp16', 
            '--gpus', '1',
            '--train_batch_size', '1',
            '--accumulate_grad_batches', '1',
            '--max_epochs', '3',
            '--enable_checkpointing', 'False',
            '--checkpoint_callback', 'False'] #TODO - Lightning throws an error when trying to checkpoint because the 'em' metric is not being returned...

#print(sys.argv)

Parse the command line arguments.


In [ ]:
args = parser.parse_args()

args.actor_handles = []

In [ ]:
args.enable_checkpointing

False

Make the output directory.

In [ ]:

Path(args.output_dir).mkdir(exist_ok=True)


## 3.2. Load Retriever & Generator

**RagConfig**

Might be interesting to see what this actually gets used for...

In [ ]:
# Load the config object        
config = RagConfig.from_pretrained('facebook/rag-token-base')


https://huggingface.co/facebook/rag-token-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpf8fr2gxs


Downloading:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/rag-token-base/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/f78a22d946ffd431f4614fef3a01644f384ab75473d1b6818dd3f8d86bc0e81c.9f453e8c75101a3b2565cc7fcf01d9ef0ab1784469dbdffce0027e1518210361
creating metadata file for /root/.cache/huggingface/transformers/f78a22d946ffd431f4614fef3a01644f384ab75473d1b6818dd3f8d86bc0e81c.9f453e8c75101a3b2565cc7fcf01d9ef0ab1784469dbdffce0027e1518210361
loading configuration file https://huggingface.co/facebook/rag-token-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f78a22d946ffd431f4614fef3a01644f384ab75473d1b6818dd3f8d86bc0e81c.9f453e8c75101a3b2565cc7fcf01d9ef0ab1784469dbdffce0027e1518210361
Model config RagConfig {
  "architectures": [
    "RagTokenForGeneration"
  ],
  "dataset": "wiki_dpr",
  "dataset_split": "train",
  "do_deduplication": true,
  "do_marginalize": false,
  "doc_sep": " // ",
  "exclude_bos_score": false,
  "forced_eos_token_id": 2

In [ ]:
config.label_smoothing = args.label_smoothing

# set extra_model_params for generator configs and load_model
extra_model_params = ("encoder_layerdrop", "decoder_layerdrop", "attention_dropout", "dropout")

args, config.generator = set_extra_model_params(extra_model_params, args, config.generator)


**RagRetriever**

We'll create the retriever with our own custom dataset.

In [ ]:

# Create the retriever
retriever = RagRetriever.from_pretrained(
    
    "facebook/rag-token-nq", # The specific pre-trained model we'll use. 

    use_dummy_dataset = False, # The dummy dataset is reduced-size Wikipedia,

    indexed_dataset = got_dataset, # Pass in our own GoT dataset!

    index_name = "embeddings", # Specify the name of the FAISS index we created.
)


https://huggingface.co/facebook/rag-token-nq/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmph7dyifbc


Downloading:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/rag-token-nq/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6337b0203e20d15c98f5e500e1e673c74e71bb8617b2753a53663b9b8e6dfc1a.59948e1fef260da10a0cecb8b6862373c32f40001848a63f985ab4f9d787f3f1
creating metadata file for /root/.cache/huggingface/transformers/6337b0203e20d15c98f5e500e1e673c74e71bb8617b2753a53663b9b8e6dfc1a.59948e1fef260da10a0cecb8b6862373c32f40001848a63f985ab4f9d787f3f1
loading configuration file https://huggingface.co/facebook/rag-token-nq/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6337b0203e20d15c98f5e500e1e673c74e71bb8617b2753a53663b9b8e6dfc1a.59948e1fef260da10a0cecb8b6862373c32f40001848a63f985ab4f9d787f3f1
/usr/local/lib/python3.7/dist-packages/transformers/models/bart/configuration_bart.py:180: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  f"Please make sure

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/rag-token-nq/resolve/main/question_encoder_tokenizer/vocab.txt in cache at /root/.cache/huggingface/transformers/26cf899a0974235af1f84469ddd94d2ee83c803c23ecead93b511ce8a0744f5c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/26cf899a0974235af1f84469ddd94d2ee83c803c23ecead93b511ce8a0744f5c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/facebook/rag-token-nq/resolve/main/question_encoder_tokenizer/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpshnhunyo


...

storing https://huggingface.co/facebook/rag-token-nq/resolve/main/generator_tokenizer/merges.txt in cache at /root/.cache/huggingface/transformers/f5fdd0c4b41c985f791c9a230cdf051da1cf9480d84e0f5f617667760a24f50f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/f5fdd0c4b41c985f791c9a230cdf051da1cf9480d84e0f5f617667760a24f50f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/facebook/rag-token-nq/resolve/main/generator_tokenizer/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpg_s1087s


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

storing https://huggingface.co/facebook/rag-token-nq/resolve/main/generator_tokenizer/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/73d054acda2f2b44f075a0d391ed392e6c53f78356bbf279581786ae35a79e4d.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0
creating metadata file for /root/.cache/huggingface/transformers/73d054acda2f2b44f075a0d391ed392e6c53f78356bbf279581786ae35a79e4d.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0
https://huggingface.co/facebook/rag-token-nq/resolve/main/generator_tokenizer/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpe15fouz9


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

storing https://huggingface.co/facebook/rag-token-nq/resolve/main/generator_tokenizer/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/a2f0711fd4c199beaf3ff2626c4a723e107e79a8d065099a3d24e7aa361c0f18.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8
creating metadata file for /root/.cache/huggingface/transformers/a2f0711fd4c199beaf3ff2626c4a723e107e79a8d065099a3d24e7aa361c0f18.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8
loading file https://huggingface.co/facebook/rag-token-nq/resolve/main/generator_tokenizer/vocab.json from cache at /root/.cache/huggingface/transformers/786598a0d343d4afb34b4f1ee17c14b58fb129abb0b3db386587bea52ffcfb11.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
loading file https://huggingface.co/facebook/rag-token-nq/resolve/main/generator_tokenizer/merges.txt from cache at /root/.cache/huggingface/transformers/f5fdd0c4b41c985f791c9a230cdf051da1cf9480d84e0f5f617667760a24f50f.5d12962c5ee615

**RagTokenForGeneration** 

Load the generator!

In [ ]:

# Load the model!
model = RagTokenForGeneration.from_pretrained(
    "facebook/rag-token-nq", 
    config = config, 
    retriever = retriever
)


https://huggingface.co/facebook/rag-token-nq/resolve/main/pytorch_model.bin not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp6tiqvkw3


Downloading:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

storing https://huggingface.co/facebook/rag-token-nq/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/6288b70ae87b5989f6801120b25a2f6ccf2e88c2f622d94805717b5b043ff71b.bceb1ea6e95a1b39d0a717f315800a39b0847e5cdeeb42615fb498775f632ee6
creating metadata file for /root/.cache/huggingface/transformers/6288b70ae87b5989f6801120b25a2f6ccf2e88c2f622d94805717b5b043ff71b.bceb1ea6e95a1b39d0a717f315800a39b0847e5cdeeb42615fb498775f632ee6
loading weights file https://huggingface.co/facebook/rag-token-nq/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/6288b70ae87b5989f6801120b25a2f6ccf2e88c2f622d94805717b5b043ff71b.bceb1ea6e95a1b39d0a717f315800a39b0847e5cdeeb42615fb498775f632ee6
Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- 

**RagTokenizer**

In [ ]:
# Load the tokenizer!
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")

loading configuration file https://huggingface.co/facebook/rag-token-nq/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6337b0203e20d15c98f5e500e1e673c74e71bb8617b2753a53663b9b8e6dfc1a.59948e1fef260da10a0cecb8b6862373c32f40001848a63f985ab4f9d787f3f1
/usr/local/lib/python3.7/dist-packages/transformers/models/bart/configuration_bart.py:180: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  f"Please make sure the config includes `forced_bos_token_id={self.bos_token_id}` in future versions. "
Model config RagConfig {
  "architectures": [
    "RagTokenForGeneration"
  ],
  "bad_words_ids": [
    [
      0,
      0
    ]
  ],
  "bos_token_id": 0,
  "dataset": "wiki_dpr",
  "dataset_split": "train",
  "decoder_start_token_id": 2,
  "do_deduplication": true,
  "do_marginalize": false,
  "doc_sep": " // ",
  "eos_token_id": 2,
  "exclude_bos_score": false,
  

### 3.2.1. Try a Question

In [ ]:
import time

t0 = time.time()

question = "Who is Arya's father?"

# Tokenize the question.
input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]

# Give the question to RAG and have it generate an answer!
generated = model.generate(input_ids)

# Convert the answer tokens back into a single string.
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

print ("Q: " + question)
print ("A: " + generated_string)

print('\nResponse took %.2f seconds' % (time.time() - t0))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1590: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=max_length)])` instead.
  UserWarning,


Q: Who is Arya's father?
A:  lord eddard stark

Response took 8.94 seconds


## 3.3. Wrap with "GenerativeQAModule"

Initialize the model object. This will do all of the downloading of the model parameters and dataset!

In [ ]:
model: GenerativeQAModule = GenerativeQAModule(args, config, tokenizer, model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/parsing.py:245: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."


# S4. Fine-Tuning

## 4.1. Responses before tuning

In [ ]:
import time


questions = [
    "Tommen’s cat was named _____.", # Ser Pounce
    "What was the name of Tommen's cat?",
    
    "Tyrion sent Janos Slynt to _____.", # The Night's Watch
    "Where did Tyrion send Janos Slynt?",

    "_____ is the name of Stannis’ sword.", # Lightbringer
    "What is the name of Stannis' sword?",

    "_____ saved Sansa from the mob in King’s Landing?", # The Hound
    "Who saved Sansa from the mob in King's Landing?",

    "Mother of Robb Stark.", # Catelyn Stark
    "Who is Robb Stark's mother?",

    "The House of Black and White is a temple located in _____ dedicated to the Many-Faced God.", # Bravos. (incorrect spelling. Should be "Braavos")
    "The House of Black and White is a temple dedicated to the Many-Faced God in which city?",

    "King asked Ned to take the title _____.", # Hand of the King.
    "What title was Ned asked to take by the King?"

    "Who is Arya's father?",
    "Where is Theon Greyjoy from?",
    "What is the title of the first episode?",
    "Who succeeded Robert Baratheon on the Iron Throne?",
    "At the beginning of the series, how many children do Ned and Catelyn Stark have?",
    "Who is the first character in the series to be called 'King in the North'?",
    "Which cast member did Kit Harington marry in 2018?",
    "How does Daenerys hatch her dragon eggs?",
    "What is the name of Arya's direwolf?",
    "Which house has a rose as its sigil?",

    "Who is Robb Stark's mom?",
    "Who does Robb Stark call his mother?",
    "What weapon does Stannis carry around?",
    "Is Stannis' sword named Widow's Wail?",
    "______ is the name of Tommen's cat, but not the name of Arya's sword.",
    "Why does Hodor say 'Hodor'?"

    "After capturing Mareena, why doesn't Daenerys continue on to conquer King's Landing?",
    "What is the name of Tommen's cat, but not the name of Arya's sword?",
    "Does Arya own a sword named 'Needle'?",

]


prev_answers = []

for question in questions:

    t0 = time.time()

    # Tokenize the question.
    input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]

    # Give the question to RAG and have it generate an answer!
    generated = model.model.generate(input_ids)

    # Convert the answer tokens back into a single string.
    generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

    print ("\nQ: " + question)
    print ("A: " + generated_string)

    prev_answers.append(generated_string)

    #print('\nResponse took %.2f seconds' % (time.time() - t0))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1590: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=max_length)])` instead.
  UserWarning,



Q: After capturing Mareena, why doesn't Daenerys continue on to conquer King's Landing?
A:  to avoid bloodshed

Q: What is the name of Tommen's cat, but not the name of Arya's sword?
A:  needle

Q: Does Arya own a sword named 'Needle'?
A:  given to her by jon


In [ ]:
print(type(tokenizer))
print(type(tokenizer.generator))
print(type(tokenizer.question_encoder))

<class 'transformers.models.rag.tokenization_rag.RagTokenizer'>
<class 'transformers.models.bart.tokenization_bart_fast.BartTokenizerFast'>
<class 'transformers.models.dpr.tokenization_dpr_fast.DPRQuestionEncoderTokenizerFast'>


## 4.2. Logging and Callbacks

In [ ]:
dataset = Path(args.data_dir).name

In [ ]:
dataset

'data'

Set up logging...

In [ ]:
args.logger_name

'default'

In [ ]:

if (
    args.logger_name == "default"
    or args.fast_dev_run
    or str(args.output_dir).startswith("/tmp")
    or str(args.output_dir).startswith("/var")
):
    training_logger = True  # don't pollute wandb logs unnecessarily

elif args.logger_name == "wandb":
    from pytorch_lightning.loggers import WandbLogger

    project = os.environ.get("WANDB_PROJECT", dataset)
    training_logger = WandbLogger(name=model.output_dir.name, project=project)


elif args.logger_name == "wandb_shared":
    from pytorch_lightning.loggers import WandbLogger

    training_logger = WandbLogger(name=model.output_dir.name, project=f"hf_{dataset}")



Early stopping callback.

In [ ]:

es_callback = (
    get_early_stopping_callback(model.val_metric, args.early_stopping_patience)
    if args.early_stopping_patience >= 0
    else False
)


Create a PyTorch Lightning trainer object.

In [ ]:
args.profile

False

In [ ]:
import logging
logger = logging.getLogger('my_logger')

# Logging levels:
#   DEBUG
#   INFO
#   WARNING
#   ERROR
#   CRITICAL

logging.basicConfig(
    level=logging.INFO 
    )

## 4.3. Tweak Arguments

In [ ]:
import transformers
transformers.__file__

'/usr/local/lib/python3.7/dist-packages/transformers/__init__.py'

In [ ]:
#!cd /usr/local/lib/python3.7/dist-packages/transformers/models/rag/

# retrieval_rag.py

In [ ]:
#print(args.train_batch_size)

#args.enable_checkpointing = False
#model.hparams.enable_checkpointing = False

if False:
    args.train_batch_size = 8
    model.hparams.train_batch_size = 8
    args.accumulate_grad_batches = 1
    model.hparams.accumulate_grad_batches = 1
    model.hparams.max_epochs = 100

In [ ]:
model.hparams

"accelerator":                       None
"accumulate_grad_batches":           1
"actor_handles":                     []
"adafactor":                         False
"adam_epsilon":                      1e-08
"amp_backend":                       native
"amp_level":                         None
"attention_dropout":                 None
"auto_lr_find":                      False
"auto_scale_batch_size":             False
"auto_select_gpus":                  False
"benchmark":                         None
"cache_dir":                         
"check_val_every_n_epoch":           1
"checkpoint_callback":               False
"config_name":                       
"data_dir":                          ./data
"decoder_layerdrop":                 None
"default_root_dir":                  None
"detect_anomaly":                    False
"deterministic":                     False
"devices":                           None
"distributed_port":                  -1
"distributed_retriever":             pyt

In [ ]:
args.do_train

False

## 4.4. Create Trainer Object

This `generic_train` function comes from the transformers library [here](https://github.com/huggingface/transformers/blob/main/examples/legacy/pytorch-lightning/lightning_base.py)

In [ ]:
# Let's just create the trainer in this step.
args.do_train = False

# Create a Trainer object.
trainer: pl.Trainer = generic_train(
    model,
    
    args,
    
    logging_callback = Seq2SeqLoggingCallback(),
    
    checkpoint_callback = get_checkpoint_callback(args.output_dir, model.val_metric),
    
    early_stopping_callback = es_callback,
    
    logger = training_logger,
    
    #custom_ddp_plugin=CustomDDP() if args.gpus > 1 else None,
    custom_ddp_plugin = None,
    
    profiler=pl.profiler.AdvancedProfiler() if args.profile else None,
)


Global seed set to 42
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:440: UserWarning: The flag `devices=auto` will be ignored, instead the device specific number 1 will be used
  f"The flag `devices={devices}` will be ignored, "
Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:172: LightningDeprecationWarning: Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instea

## 4.5. Run Training!

Now let's do the training!

In [ ]:
trainer.fit(model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:377: LightningDeprecationWarning: The `Callback.on_batch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `Callback.on_train_batch_end` instead.
  f"The `Callback.{hook}` hook was deprecated in v1.6 and"
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the Py

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1590: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=max_length)])` instead.
  UserWarning,


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# S5. Testing and Evaluation

## 5.1. trainer.test

In [ ]:
# Note: there is an argument, "args.do_predict:" intended to govern this.
# Note: test() without a model tests using the best checkpoint automatically
trainer.test(model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:377: LightningDeprecationWarning: The `Callback.on_batch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `Callback.on_train_batch_end` instead.
  f"The `Callback.{hook}` hook was deprecated in v1.6 and"
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1590: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=max_length)])` instead.
  UserWarning,
INFO:my_logger:***** test results at step 00342 *****


[{}]


## 5.2. Ask Questions

In [ ]:
import time

t0 = time.time()

question = "Who is Arya's father?"

# Tokenize the question.
input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]

# Give the question to RAG and have it generate an answer!
generated = model.model.generate(input_ids)

# Convert the answer tokens back into a single string.
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

print ("Q: " + question)
print ("A: " + generated_string)

print('\nResponse took %.2f seconds' % (time.time() - t0))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1590: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=max_length)])` instead.
  UserWarning,


Q: Who is Arya's father?
A: Lord Edwin Lannister.

Response took 10.23 seconds


In [ ]:
import time

t0 = time.time()

question = "Who is Arya's father?"

# Tokenize the question.
input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]

# Give the question to RAG and have it generate an answer!
# 
generated = model.model.generate(input_ids, num_beams=4, max_length=5)

# Convert the answer tokens back into a single string.
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

print ("Q: " + question)
print ("A: " + generated_string)

print('\nResponse took %.2f seconds' % (time.time() - t0))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2097: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  UserWarning,


In [ ]:
import time

new_answers = []

for question in questions:

    t0 = time.time()

    # Tokenize the question.
    input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]

    # Give the question to RAG and have it generate an answer!
    generated = model.model.generate(input_ids)

    # Convert the answer tokens back into a single string.
    generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

    print ("\nQ: " + question)
    print ("A: " + generated_string)

    new_answers.append(generated_string)

    #print('\nResponse took %.2f seconds' % (time.time() - t0))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1590: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=max_length)])` instead.
  UserWarning,



Q: Tommen’s cat was named _____.
A: Brynden.

Q: What was the name of Tommen's cat?
A: Ser Pounce.

Q: Tyrion sent Janos Slynt to _____.
A: The Night’s Watch.

Q: Where did Tyrion send Janos Slynt?
A: The Great Pyramid

Q: _____ is the name of Stannis’ sword.
A: Widow’s Wail.

Q: What is the name of Stannis' sword?
A: The Great Sealord.

Q: _____ saved Sansa from the mob in King’s Landing?
A: Brienne of Tarth.

Q: Who saved Sansa from the mob in King's Landing?
A: Brienne.

Q: Mother of Robb Stark.
A: Catelyn Stark.

Q: Who is Robb Stark's mother?
A: Catelyn Stark.

Q: The House of Black and White is a temple located in _____ dedicated to the Many-Faced God.
A: Bravos.

Q: The House of Black and White is a temple dedicated to the Many-Faced God in which city?
A: Belfast

Q: King asked Ned to take the title _____.
A: The King’s Hand.

Q: What title was Ned asked to take by the King?Who is Arya's father?
A: Hand of the King.

Q: Where is Theon Greyjoy from?
A: The Night’s Watch.

Q: Wha

In [ ]:
print(questions)


['Tommen’s cat was named _____.', "What was the name of Tommen's cat?", 'Tyrion sent Janos Slynt to _____.', 'Where did Tyrion send Janos Slynt?', '_____ is the name of Stannis’ sword.', "What is the name of Stannis' sword?", '_____ saved Sansa from the mob in King’s Landing?', "Who saved Sansa from the mob in King's Landing?", 'Mother of Robb Stark.', "Who is Robb Stark's mother?", 'The House of Black and White is a temple located in _____ dedicated to the Many-Faced God.', 'The House of Black and White is a temple dedicated to the Many-Faced God in which city?', 'King asked Ned to take the title _____.', "What title was Ned asked to take by the King?Who is Arya's father?", 'Where is Theon Greyjoy from?', 'What is the title of the first episode?', 'Who succeeded Robert Baratheon on the Iron Throne?', 'At the beginning of the series, how many children do Ned and Catelyn Stark have?', "Who is the first character in the series to be called 'King in the North'?", 'Which cast member did Ki

In [ ]:
print(prev_answers)

[' ser pounce', ' ser pounce', ' mance', ' mance', ' hearts', ' the', ' tyrion', ' theon', ' catelyn stark', ' lady catelyn', ' braavos', ' braavos', ' the', ' hand of the king', ' the iron islands', ' winter is coming', '', ' five', ' robin', ' rose leslie', ' ancient fossilized dragon eggs', ' nymer', ' house of']


In [ ]:
print(new_answers)

['Brynden.', 'Ser Pounce.', 'The Night’s Watch.', 'The Great Pyramid', 'Widow’s Wail.', 'The Great Sealord.', 'Brienne of Tarth.', 'Brienne.', 'Catelyn Stark.', 'Catelyn Stark.', 'Bravos.', 'Belfast', 'The King’s Hand.', 'Hand of the King.', 'The Night’s Watch.', 'The Night’s Watch.', 'Jorah Mormont.', 'Three.', 'Robb Stark.', 'Melisandre.', 'Hatching.', 'Nymeria.', 'House Greyjoy.']


In [ ]:
for q in new_answers:
    print(q)

Brynden.
Ser Pounce.
The Night’s Watch.
The Great Pyramid
Widow’s Wail.
The Great Sealord.
Brienne of Tarth.
Brienne.
Catelyn Stark.
Catelyn Stark.
Bravos.
Belfast
The King’s Hand.
Hand of the King.
The Night’s Watch.
The Night’s Watch.
Jorah Mormont.
Three.
Robb Stark.
Melisandre.
Hatching.
Nymeria.
House Greyjoy.


In [ ]:
type(model.tokenizer)

transformers.models.rag.tokenization_rag.RagTokenizer

In [ ]:
type(model.tokenizer.generator)

transformers.models.bart.tokenization_bart_fast.BartTokenizerFast

# Appendix

## A1. Save

In [ ]:
model.save()

AttributeError: ignored

In [ ]:

pickle_save(model.hparams, model.output_dir / "hparams.pkl")


## A2. Answers before fine-tuning

In [ ]:
import time

questions = [
    "Mother of Robb Stark.", # Catelyn Stark
    "Who is Robb Stark's mother?",

    "The House of Black and White is a temple located in _____ dedicated to the Many-Faced God.", # Bravos. (incorrect spelling. Should be "Braavos")
    "The House of Black and White is a temple dedicated to the Many-Faced God in which city?",

    "King asked Ned to take the title _____.", # Hand of the King.
    "What title was Ned asked to take by the King?"
]

for question in questions:

    t0 = time.time()

    # Tokenize the question.
    input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]

    # Give the question to RAG and have it generate an answer!
    generated = model.model.generate(input_ids)

    # Convert the answer tokens back into a single string.
    generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

    print ("\nQ: " + question)
    print ("A: " + generated_string)

    #print('\nResponse took %.2f seconds' % (time.time() - t0))

In [ ]:
import time

questions = [
    "Who is Arya's father?",
    "Where is Theon Greyjoy from?",
    "What is the title of the first episode?",
    "Who succeeded Robert Baratheon on the Iron Throne?",
    "At the beginning of the series, how many children do Ned and Catelyn Stark have?",
    "Who is the first character in the series to be called 'King in the North'?",
    "Which cast member did Kit Harington marry in 2018?",
    "How does Daenerys hatch her dragon eggs?",
    "What is the name of Arya's direwolf?",
    "Which house has a rose as its sigil?"
]

for question in questions:

    t0 = time.time()

    # Tokenize the question.
    input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]

    # Give the question to RAG and have it generate an answer!
    generated = model.model.generate(input_ids)

    # Convert the answer tokens back into a single string.
    generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

    print ("\nQ: " + question)
    print ("A: " + generated_string)

    #print('\nResponse took %.2f seconds' % (time.time() - t0))

## A2. Script Arguments

```
usage: finetune_rag.py [-h] [--logger [LOGGER]]
                       [--checkpoint_callback [CHECKPOINT_CALLBACK]]
                       [--enable_checkpointing [ENABLE_CHECKPOINTING]]
                       [--default_root_dir DEFAULT_ROOT_DIR]
                       [--gradient_clip_val GRADIENT_CLIP_VAL]
                       [--gradient_clip_algorithm GRADIENT_CLIP_ALGORITHM]
                       [--process_position PROCESS_POSITION]
                       [--num_nodes NUM_NODES] [--num_processes NUM_PROCESSES]
                       [--devices DEVICES] [--gpus GPUS]
                       [--auto_select_gpus [AUTO_SELECT_GPUS]]
                       [--tpu_cores TPU_CORES] [--ipus IPUS]
                       [--log_gpu_memory LOG_GPU_MEMORY]
                       [--progress_bar_refresh_rate PROGRESS_BAR_REFRESH_RATE]
                       [--enable_progress_bar [ENABLE_PROGRESS_BAR]]
                       [--overfit_batches OVERFIT_BATCHES]
                       [--track_grad_norm TRACK_GRAD_NORM]
                       [--check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH]
                       [--fast_dev_run [FAST_DEV_RUN]]
                       [--accumulate_grad_batches ACCUMULATE_GRAD_BATCHES]
                       [--max_epochs MAX_EPOCHS] [--min_epochs MIN_EPOCHS]
                       [--max_steps MAX_STEPS] [--min_steps MIN_STEPS]
                       [--max_time MAX_TIME]
                       [--limit_train_batches LIMIT_TRAIN_BATCHES]
                       [--limit_val_batches LIMIT_VAL_BATCHES]
                       [--limit_test_batches LIMIT_TEST_BATCHES]
                       [--limit_predict_batches LIMIT_PREDICT_BATCHES]
                       [--val_check_interval VAL_CHECK_INTERVAL]
                       [--flush_logs_every_n_steps FLUSH_LOGS_EVERY_N_STEPS]
                       [--log_every_n_steps LOG_EVERY_N_STEPS]
                       [--accelerator ACCELERATOR] [--strategy STRATEGY]
                       [--sync_batchnorm [SYNC_BATCHNORM]]
                       [--precision PRECISION]
                       [--enable_model_summary [ENABLE_MODEL_SUMMARY]]
                       [--weights_summary WEIGHTS_SUMMARY]
                       [--weights_save_path WEIGHTS_SAVE_PATH]
                       [--num_sanity_val_steps NUM_SANITY_VAL_STEPS]
                       [--resume_from_checkpoint RESUME_FROM_CHECKPOINT]
                       [--profiler PROFILER] [--benchmark [BENCHMARK]]
                       [--deterministic [DETERMINISTIC]]
                       [--reload_dataloaders_every_n_epochs RELOAD_DATALOADERS_EVERY_N_EPOCHS]
                       [--auto_lr_find [AUTO_LR_FIND]]
                       [--replace_sampler_ddp [REPLACE_SAMPLER_DDP]]
                       [--detect_anomaly [DETECT_ANOMALY]]
                       [--auto_scale_batch_size [AUTO_SCALE_BATCH_SIZE]]
                       [--prepare_data_per_node [PREPARE_DATA_PER_NODE]]
                       [--plugins PLUGINS] [--amp_backend AMP_BACKEND]
                       [--amp_level AMP_LEVEL]
                       [--move_metrics_to_cpu [MOVE_METRICS_TO_CPU]]
                       [--multiple_trainloader_mode MULTIPLE_TRAINLOADER_MODE]
                       [--stochastic_weight_avg [STOCHASTIC_WEIGHT_AVG]]
                       [--terminate_on_nan [TERMINATE_ON_NAN]]
                       --model_name_or_path MODEL_NAME_OR_PATH
                       [--config_name CONFIG_NAME]
                       [--tokenizer_name TOKENIZER_NAME]
                       [--cache_dir CACHE_DIR]
                       [--encoder_layerdrop ENCODER_LAYERDROP]
                       [--decoder_layerdrop DECODER_LAYERDROP]
                       [--dropout DROPOUT]
                       [--attention_dropout ATTENTION_DROPOUT]
                       [--learning_rate LEARNING_RATE]
                       [--lr_scheduler {cosine, cosine_w_restarts, linear, polynomial}]
                       [--weight_decay WEIGHT_DECAY]
                       [--adam_epsilon ADAM_EPSILON]
                       [--warmup_steps WARMUP_STEPS]
                       [--num_workers NUM_WORKERS]
                       [--num_train_epochs MAX_EPOCHS]
                       [--train_batch_size TRAIN_BATCH_SIZE]
                       [--eval_batch_size EVAL_BATCH_SIZE] [--adafactor]
                       --output_dir OUTPUT_DIR [--fp16]
                       [--fp16_opt_level FP16_OPT_LEVEL]
                       [--n_tpu_cores TPU_CORES]
                       [--max_grad_norm GRADIENT_CLIP_VAL] [--do_train]
                       [--do_predict]
                       [--gradient_accumulation_steps ACCUMULATE_GRAD_BATCHES]
                       [--seed SEED] --data_dir DATA_DIR
                       [--max_source_length MAX_SOURCE_LENGTH]
                       [--max_target_length MAX_TARGET_LENGTH]
                       [--val_max_target_length VAL_MAX_TARGET_LENGTH]
                       [--test_max_target_length TEST_MAX_TARGET_LENGTH]
                       [--logger_name {default,wandb,wandb_shared}]
                       [--n_train N_TRAIN] [--n_val N_VAL] [--n_test N_TEST]
                       [--label_smoothing LABEL_SMOOTHING] [--prefix PREFIX]
                       [--early_stopping_patience EARLY_STOPPING_PATIENCE]
                       [--distributed-port DISTRIBUTED_PORT]
                       [--model_type {rag_sequence,rag_token,bart,t5}]
                       [--index_name INDEX_NAME]
                       [--passages_path PASSAGES_PATH]
                       [--index_path INDEX_PATH]
                       [--distributed_retriever {ray,pytorch}]
                       [--use_dummy_dataset USE_DUMMY_DATASET]
                       [--ray-address RAY_ADDRESS]
                       [--num_retrieval_workers NUM_RETRIEVAL_WORKERS]
                       [--profile]

optional arguments:
  -h, --help            show this help message and exit
  --model_name_or_path MODEL_NAME_OR_PATH
                        Path to pretrained model or model identifier from
                        huggingface.co/models
  --config_name CONFIG_NAME
                        Pretrained config name or path if not the same as
                        model_name
  --tokenizer_name TOKENIZER_NAME
                        Pretrained tokenizer name or path if not the same as
                        model_name
  --cache_dir CACHE_DIR
                        Where do you want to store the pre-trained models
                        downloaded from huggingface.co
  --encoder_layerdrop ENCODER_LAYERDROP
                        Encoder layer dropout probability (Optional). Goes
                        into model.config
  --decoder_layerdrop DECODER_LAYERDROP
                        Decoder layer dropout probability (Optional). Goes
                        into model.config
  --dropout DROPOUT     Dropout probability (Optional). Goes into model.config
  --attention_dropout ATTENTION_DROPOUT
                        Attention dropout probability (Optional). Goes into
                        model.config
  --learning_rate LEARNING_RATE
                        The initial learning rate for Adam.
  --lr_scheduler {cosine, cosine_w_restarts, linear, polynomial}
                        Learning rate scheduler
  --weight_decay WEIGHT_DECAY
                        Weight decay if we apply some.
  --adam_epsilon ADAM_EPSILON
                        Epsilon for Adam optimizer.
  --warmup_steps WARMUP_STEPS
                        Linear warmup over warmup_steps.
  --num_workers NUM_WORKERS
                        kwarg passed to DataLoader
  --num_train_epochs MAX_EPOCHS
  --train_batch_size TRAIN_BATCH_SIZE
  --eval_batch_size EVAL_BATCH_SIZE
  --adafactor
  --output_dir OUTPUT_DIR
                        The output directory where the model predictions and
                        checkpoints will be written.
  --fp16                Whether to use 16-bit (mixed) precision (through
                        NVIDIA apex) instead of 32-bit
  --fp16_opt_level FP16_OPT_LEVEL
                        For fp16: Apex AMP optimization level selected in
                        ['O0', 'O1', 'O2', and 'O3'].See details at
                        https://nvidia.github.io/apex/amp.html
  --n_tpu_cores TPU_CORES
  --max_grad_norm GRADIENT_CLIP_VAL
                        Max gradient norm
  --do_train            Whether to run training.
  --do_predict          Whether to run predictions on the test set.
  --gradient_accumulation_steps ACCUMULATE_GRAD_BATCHES
                        Number of updates steps to accumulate before
                        performing a backward/update pass.
  --seed SEED           random seed for initialization
  --data_dir DATA_DIR   The input data dir. Should contain the training files
                        for the CoNLL-2003 NER task.
  --max_source_length MAX_SOURCE_LENGTH
                        The maximum total input sequence length after
                        tokenization. Sequences longer than this will be
                        truncated, sequences shorter will be padded.
  --max_target_length MAX_TARGET_LENGTH
                        The maximum total input sequence length after
                        tokenization. Sequences longer than this will be
                        truncated, sequences shorter will be padded.
  --val_max_target_length VAL_MAX_TARGET_LENGTH
                        The maximum total input sequence length after
                        tokenization. Sequences longer than this will be
                        truncated, sequences shorter will be padded.
  --test_max_target_length TEST_MAX_TARGET_LENGTH
                        The maximum total input sequence length after
                        tokenization. Sequences longer than this will be
                        truncated, sequences shorter will be padded.
  --logger_name {default,wandb,wandb_shared}
  --n_train N_TRAIN     # examples. -1 means use all.
  --n_val N_VAL         # examples. -1 means use all.
  --n_test N_TEST       # examples. -1 means use all.
  --label_smoothing LABEL_SMOOTHING
  --prefix PREFIX       Prefix added at the beginning of each text, typically
                        used with T5-based models.
  --early_stopping_patience EARLY_STOPPING_PATIENCE
                        -1 means never early stop. early_stopping_patience is
                        measured in validation checks, not epochs. So
                        val_check_interval will effect it.
  --distributed-port DISTRIBUTED_PORT
                        Port number for distributed training.
  --model_type {rag_sequence,rag_token,bart,t5}
                        RAG model type: sequence or token, if none specified,
                        the type is inferred from the model_name_or_path
  --index_name INDEX_NAME
                        Name of the index to use: 'hf' for a canonical dataset
                        from the datasets library (default), 'custom' for a
                        local index, or 'legacy' for the orignal one)
  --passages_path PASSAGES_PATH
                        Path to the dataset of passages for custom index. More
                        info about custom indexes in the RagRetriever
                        documentation as well as in
                        `examples/rag/use_own_knowledge_dataset.py`
  --index_path INDEX_PATH
                        Path to the faiss index for custom index. More info
                        about custom indexes in the RagRetriever documentation
                        as well as in
                        `examples/rag/use_own_knowledge_dataset.py`
  --distributed_retriever {ray,pytorch}
                        What implementation to use for distributed retriever?
                        If pytorch is selected, the index is loaded on
                        training worker 0, and torch.distributed is used to
                        handle communication between training worker 0, and
                        the other training workers. If ray is selected, the
                        Ray library is used to create load the index on
                        separate processes, and Ray handles the communication
                        between the training workers and the retrieval actors.
  --use_dummy_dataset USE_DUMMY_DATASET
                        Whether to use the dummy version of the dataset index.
                        More info about custom indexes in the RagRetriever
                        documentation as well as in
                        `examples/rag/use_own_knowledge_dataset.py`
  --ray-address RAY_ADDRESS
                        The address of the Ray cluster to connect to. If not
                        specified, Ray will attempt to automatically detect
                        the cluster. Has no effect if pytorch is used as the
                        distributed retriever.
  --num_retrieval_workers NUM_RETRIEVAL_WORKERS
                        The number of retrieval actors to use when Ray is
                        selectedfor the distributed retriever. Has no effect
                        when distributed_retriever is set to pytorch.
  --profile             If True, use
                        pytorch_lightning.profiler.AdvancedProfiler to profile
                        the Trainer.

pl.Trainer:
  --logger [LOGGER]     Logger (or iterable collection of loggers) for
                        experiment tracking. A ``True`` value uses the default
                        ``TensorBoardLogger``. ``False`` will disable logging.
                        If multiple loggers are provided and the `save_dir`
                        property of that logger is not set, local files
                        (checkpoints, profiler traces, etc.) are saved in
                        ``default_root_dir`` rather than in the ``log_dir`` of
                        any of the individual loggers. Default: ``True``.
  --checkpoint_callback [CHECKPOINT_CALLBACK]
                        If ``True``, enable checkpointing. Default: ``None``.
                        .. deprecated:: v1.5 ``checkpoint_callback`` has been
                        deprecated in v1.5 and will be removed in v1.7. Please
                        consider using ``enable_checkpointing`` instead.
  --enable_checkpointing [ENABLE_CHECKPOINTING]
                        If ``True``, enable checkpointing. It will configure a
                        default ModelCheckpoint callback if there is no user-
                        defined ModelCheckpoint in :paramref:`~pytorch_lightni
                        ng.trainer.trainer.Trainer.callbacks`. Default:
                        ``True``.
  --default_root_dir DEFAULT_ROOT_DIR
                        Default path for logs and weights when no
                        logger/ckpt_callback passed. Default: ``os.getcwd()``.
                        Can be remote file paths such as `s3://mybucket/path`
                        or 'hdfs://path/'
  --gradient_clip_val GRADIENT_CLIP_VAL
                        The value at which to clip gradients. Passing
                        ``gradient_clip_val=None`` disables gradient clipping.
                        If using Automatic Mixed Precision (AMP), the
                        gradients will be unscaled before. Default: ``None``.
  --gradient_clip_algorithm GRADIENT_CLIP_ALGORITHM
                        The gradient clipping algorithm to use. Pass
                        ``gradient_clip_algorithm="value"`` to clip by value,
                        and ``gradient_clip_algorithm="norm"`` to clip by
                        norm. By default it will be set to ``"norm"``.
  --process_position PROCESS_POSITION
                        Orders the progress bar when running multiple models
                        on same machine. .. deprecated:: v1.5
                        ``process_position`` has been deprecated in v1.5 and
                        will be removed in v1.7. Please pass :class:`~pytorch_
                        lightning.callbacks.progress.TQDMProgressBar` with
                        ``process_position`` directly to the Trainer's
                        ``callbacks`` argument instead.
  --num_nodes NUM_NODES
                        Number of GPU nodes for distributed training. Default:
                        ``1``.
  --num_processes NUM_PROCESSES
                        Number of processes for distributed training with
                        ``accelerator="cpu"``. Default: ``1``.
  --devices DEVICES     Will be mapped to either `gpus`, `tpu_cores`,
                        `num_processes` or `ipus`, based on the accelerator
                        type.
  --gpus GPUS           Number of GPUs to train on (int) or which GPUs to
                        train on (list or str) applied per node Default:
                        ``None``.
  --auto_select_gpus [AUTO_SELECT_GPUS]
                        If enabled and ``gpus`` is an integer, pick available
                        gpus automatically. This is especially useful when
                        GPUs are configured to be in "exclusive mode", such
                        that only one process at a time can access them.
                        Default: ``False``.
  --tpu_cores TPU_CORES
                        How many TPU cores to train on (1 or 8) / Single TPU
                        to train on (1) Default: ``None``.
  --ipus IPUS           How many IPUs to train on. Default: ``None``.
  --log_gpu_memory LOG_GPU_MEMORY
                        None, 'min_max', 'all'. Might slow performance. ..
                        deprecated:: v1.5 Deprecated in v1.5.0 and will be
                        removed in v1.7.0 Please use the
                        ``DeviceStatsMonitor`` callback directly instead.
  --progress_bar_refresh_rate PROGRESS_BAR_REFRESH_RATE
                        How often to refresh progress bar (in steps). Value
                        ``0`` disables progress bar. Ignored when a custom
                        progress bar is passed to
                        :paramref:`~Trainer.callbacks`. Default: None, means a
                        suitable value will be chosen based on the environment
                        (terminal, Google COLAB, etc.). .. deprecated:: v1.5
                        ``progress_bar_refresh_rate`` has been deprecated in
                        v1.5 and will be removed in v1.7. Please pass :class:`
                        ~pytorch_lightning.callbacks.progress.TQDMProgressBar`
                        with ``refresh_rate`` directly to the Trainer's
                        ``callbacks`` argument instead. To disable the
                        progress bar, pass ``enable_progress_bar = False`` to
                        the Trainer.
  --enable_progress_bar [ENABLE_PROGRESS_BAR]
                        Whether to enable to progress bar by default. Default:
                        ``False``.
  --overfit_batches OVERFIT_BATCHES
                        Overfit a fraction of training data (float) or a set
                        number of batches (int). Default: ``0.0``.
  --track_grad_norm TRACK_GRAD_NORM
                        -1 no tracking. Otherwise tracks that p-norm. May be
                        set to 'inf' infinity-norm. If using Automatic Mixed
                        Precision (AMP), the gradients will be unscaled before
                        logging them. Default: ``-1``.
  --check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH
                        Check val every n train epochs. Default: ``1``.
  --fast_dev_run [FAST_DEV_RUN]
                        Runs n if set to ``n`` (int) else 1 if set to ``True``
                        batch(es) of train, val and test to find any bugs (ie:
                        a sort of unit test). Default: ``False``.
  --accumulate_grad_batches ACCUMULATE_GRAD_BATCHES
                        Accumulates grads every k batches or as set up in the
                        dict. Default: ``None``.
  --max_epochs MAX_EPOCHS
                        Stop training once this number of epochs is reached.
                        Disabled by default (None). If both max_epochs and
                        max_steps are not specified, defaults to ``max_epochs
                        = 1000``. To enable infinite training, set
                        ``max_epochs = -1``.
  --min_epochs MIN_EPOCHS
                        Force training for at least these many epochs.
                        Disabled by default (None).
  --max_steps MAX_STEPS
                        Stop training after this number of steps. Disabled by
                        default (-1). If ``max_steps = -1`` and ``max_epochs =
                        None``, will default to ``max_epochs = 1000``. To
                        enable infinite training, set ``max_epochs`` to
                        ``-1``.
  --min_steps MIN_STEPS
                        Force training for at least these number of steps.
                        Disabled by default (``None``).
  --max_time MAX_TIME   Stop training after this amount of time has passed.
                        Disabled by default (``None``). The time duration can
                        be specified in the format DD:HH:MM:SS (days, hours,
                        minutes seconds), as a :class:`datetime.timedelta`, or
                        a dictionary with keys that will be passed to
                        :class:`datetime.timedelta`.
  --limit_train_batches LIMIT_TRAIN_BATCHES
                        How much of training dataset to check (float =
                        fraction, int = num_batches). Default: ``1.0``.
  --limit_val_batches LIMIT_VAL_BATCHES
                        How much of validation dataset to check (float =
                        fraction, int = num_batches). Default: ``1.0``.
  --limit_test_batches LIMIT_TEST_BATCHES
                        How much of test dataset to check (float = fraction,
                        int = num_batches). Default: ``1.0``.
  --limit_predict_batches LIMIT_PREDICT_BATCHES
                        How much of prediction dataset to check (float =
                        fraction, int = num_batches). Default: ``1.0``.
  --val_check_interval VAL_CHECK_INTERVAL
                        How often to check the validation set. Pass a
                        ``float`` in the range [0.0, 1.0] to check after a
                        fraction of the training epoch. Pass an ``int`` to
                        check after a fixed number of training batches.
                        Default: ``1.0``.
  --flush_logs_every_n_steps FLUSH_LOGS_EVERY_N_STEPS
                        How often to flush logs to disk (defaults to every 100
                        steps). .. deprecated:: v1.5
                        ``flush_logs_every_n_steps`` has been deprecated in
                        v1.5 and will be removed in v1.7. Please configure
                        flushing directly in the logger instead.
  --log_every_n_steps LOG_EVERY_N_STEPS
                        How often to log within steps. Default: ``50``.
  --accelerator ACCELERATOR
                        Supports passing different accelerator types ("cpu",
                        "gpu", "tpu", "ipu", "hpu", "auto") as well as custom
                        accelerator instances. .. deprecated:: v1.5 Passing
                        training strategies (e.g., 'ddp') to ``accelerator``
                        has been deprecated in v1.5.0 and will be removed in
                        v1.7.0. Please use the ``strategy`` argument instead.
  --strategy STRATEGY   Supports different training strategies with aliases as
                        well custom strategies. Default: ``None``.
  --sync_batchnorm [SYNC_BATCHNORM]
                        Synchronize batch norm layers between process
                        groups/whole world. Default: ``False``.
  --precision PRECISION
                        Double precision (64), full precision (32), half
                        precision (16) or bfloat16 precision (bf16). Can be
                        used on CPU, GPU, TPUs, HPUs or IPUs. Default: ``32``.
  --enable_model_summary [ENABLE_MODEL_SUMMARY]
                        Whether to enable model summarization by default.
                        Default: ``True``.
  --weights_summary WEIGHTS_SUMMARY
                        Prints a summary of the weights when training begins.
                        .. deprecated:: v1.5 ``weights_summary`` has been
                        deprecated in v1.5 and will be removed in v1.7. To
                        disable the summary, pass ``enable_model_summary =
                        False`` to the Trainer. To customize the summary, pass
                        :class:`~pytorch_lightning.callbacks.model_summary.Mod
                        elSummary` directly to the Trainer's ``callbacks``
                        argument.
  --weights_save_path WEIGHTS_SAVE_PATH
                        Where to save weights if specified. Will override
                        default_root_dir for checkpoints only. Use this if for
                        whatever reason you need the checkpoints stored in a
                        different place than the logs written in
                        `default_root_dir`. Can be remote file paths such as
                        `s3://mybucket/path` or 'hdfs://path/' Defaults to
                        `default_root_dir`. .. deprecated:: v1.6
                        ``weights_save_path`` has been deprecated in v1.6 and
                        will be removed in v1.8. Please pass ``dirpath``
                        directly to the :class:`~pytorch_lightning.callbacks.m
                        odel_checkpoint.ModelCheckpoint` callback.
  --num_sanity_val_steps NUM_SANITY_VAL_STEPS
                        Sanity check runs n validation batches before starting
                        the training routine. Set it to `-1` to run all
                        batches in all validation dataloaders. Default: ``2``.
  --resume_from_checkpoint RESUME_FROM_CHECKPOINT
                        Path/URL of the checkpoint from which training is
                        resumed. If there is no checkpoint file at the path,
                        an exception is raised. If resuming from mid-epoch
                        checkpoint, training will start from the beginning of
                        the next epoch. .. deprecated:: v1.5
                        ``resume_from_checkpoint`` is deprecated in v1.5 and
                        will be removed in v2.0. Please pass the path to
                        ``Trainer.fit(..., ckpt_path=...)`` instead.
  --profiler PROFILER   To profile individual steps during training and assist
                        in identifying bottlenecks. Default: ``None``.
  --benchmark [BENCHMARK]
                        Sets ``torch.backends.cudnn.benchmark``. Defaults to
                        ``True`` if :paramref:`~pytorch_lightning.trainer.trai
                        ner.Trainer.deterministic` is ``False``. Overwrite to
                        manually set a different value. Default: ``None``.
  --deterministic [DETERMINISTIC]
                        If ``True``, sets whether PyTorch operations must use
                        deterministic algorithms. Default: ``False``.
  --reload_dataloaders_every_n_epochs RELOAD_DATALOADERS_EVERY_N_EPOCHS
                        Set to a non-negative integer to reload dataloaders
                        every n epochs. Default: ``0``.
  --auto_lr_find [AUTO_LR_FIND]
                        If set to True, will make trainer.tune() run a
                        learning rate finder, trying to optimize initial
                        learning for faster convergence. trainer.tune() method
                        will set the suggested learning rate in self.lr or
                        self.learning_rate in the LightningModule. To use a
                        different key set a string instead of True with the
                        key name. Default: ``False``.
  --replace_sampler_ddp [REPLACE_SAMPLER_DDP]
                        Explicitly enables or disables sampler replacement. If
                        not specified this will toggled automatically when DDP
                        is used. By default it will add ``shuffle=True`` for
                        train sampler and ``shuffle=False`` for val/test
                        sampler. If you want to customize it, you can set
                        ``replace_sampler_ddp=False`` and add your own
                        distributed sampler.
  --detect_anomaly [DETECT_ANOMALY]
                        Enable anomaly detection for the autograd engine.
                        Default: ``False``.
  --auto_scale_batch_size [AUTO_SCALE_BATCH_SIZE]
                        If set to True, will `initially` run a batch size
                        finder trying to find the largest batch size that fits
                        into memory. The result will be stored in
                        self.batch_size in the LightningModule. Additionally,
                        can be set to either `power` that estimates the batch
                        size through a power search or `binsearch` that
                        estimates the batch size through a binary search.
                        Default: ``False``.
  --prepare_data_per_node [PREPARE_DATA_PER_NODE]
                        If True, each LOCAL_RANK=0 will call prepare data.
                        Otherwise only NODE_RANK=0, LOCAL_RANK=0 will prepare
                        data .. deprecated:: v1.5 Deprecated in v1.5.0 and
                        will be removed in v1.7.0 Please set
                        ``prepare_data_per_node`` in ``LightningDataModule``
                        and/or ``LightningModule`` directly instead.
  --plugins PLUGINS     Plugins allow modification of core behavior like ddp
                        and amp, and enable custom lightning plugins. Default:
                        ``None``.
  --amp_backend AMP_BACKEND
                        The mixed precision backend to use ("native" or
                        "apex"). Default: ``'native''``.
  --amp_level AMP_LEVEL
                        The optimization level to use (O1, O2, etc...). By
                        default it will be set to "O2" if ``amp_backend`` is
                        set to "apex".
  --move_metrics_to_cpu [MOVE_METRICS_TO_CPU]
                        Whether to force internal logged metrics to be moved
                        to cpu. This can save some gpu memory, but can make
                        training slower. Use with attention. Default:
                        ``False``.
  --multiple_trainloader_mode MULTIPLE_TRAINLOADER_MODE
                        How to loop over the datasets when there are multiple
                        train loaders. In 'max_size_cycle' mode, the trainer
                        ends one epoch when the largest dataset is traversed,
                        and smaller datasets reload when running out of their
                        data. In 'min_size' mode, all the datasets reload when
                        reaching the minimum length of datasets. Default:
                        ``"max_size_cycle"``.
  --stochastic_weight_avg [STOCHASTIC_WEIGHT_AVG]
                        Whether to use `Stochastic Weight Averaging (SWA)
                        <https://pytorch.org/blog/pytorch-1.6-now-includes-
                        stochastic-weight-averaging/>`_. Default: ``False``.
                        .. deprecated:: v1.5 ``stochastic_weight_avg`` has
                        been deprecated in v1.5 and will be removed in v1.7.
                        Please pass :class:`~pytorch_lightning.callbacks.stoch
                        astic_weight_avg.StochasticWeightAveraging` directly
                        to the Trainer's ``callbacks`` argument instead.
  --terminate_on_nan [TERMINATE_ON_NAN]
                        If set to True, will terminate training (by raising a
                        `ValueError`) at the end of each training batch, if
                        any of the parameters or the loss are NaN or +/-inf.
                        .. deprecated:: v1.5 Trainer argument
                        ``terminate_on_nan`` was deprecated in v1.5 and will
                        be removed in 1.7. Please use ``detect_anomaly``
                        instead.
```                        

## A3. Pinned Revisions

**4/1/22 - Working, Pre-Rip**

* This version of the Notebook runs successfully using wikipedia. Test and validation scores are 0, though, on GoT questions.

## A4. Retrieve Script Files

The code for this example is spread across 5 different scripts.

In [ ]:
import wget 

#wget.download('https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/rag/finetune_rag.py')
#wget.download('https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/rag/callbacks_rag.py')
#wget.download('https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/rag/utils_rag.py')
#wget.download('https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/rag/distributed_pytorch_retriever.py')
#wget.download('https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/rag/lightning_base.py')
